In [ ]:
# 測試：查詢英雄分析 (以 Ahri 中路為例, champion_id=103)
print("📊 查詢 Ahri 中路的英雄分析...\n")

result = await client.get_champion_analysis(
    region="kr",
    champion_id=103,  # Ahri
    lane="MID"
)

print("結果:")
print("=" * 60)
print(result)

## 9. 測試：查詢英雄分析 (Counter/Ban/Pick)

In [ ]:
# 測試：查詢英雄列表
print("🎮 查詢英雄列表...\n")

result = await client.list_champions(region="kr")

print("結果:")
print("=" * 60)
print(result)

## 8. 測試：查詢英雄列表

In [ ]:
# 測試：查詢 Faker (Hide on bush) 的資料
print("🔍 查詢 Faker (Hide on bush) 的資料...\n")

result = await client.get_summoner_profile(
    region="kr",
    summoner_id="Hide on bush",
    game_type="SOLORANKED"
)

print("結果:")
print("=" * 60)
print(result)

## 7. 測試：查詢召喚師資料

In [ ]:
# 測試：列出所有工具
tools = await client.list_tools()

print(f"✅ 成功連接！找到 {len(tools)} 個工具\n")
print("工具列表:")
print("=" * 60)

for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}")
    if hasattr(tool, 'description'):
        print(f"   {tool.description[:100]}...")
    print()

## 6. 測試：列出所有工具

In [ ]:
class SimpleMCPClient:
    """
    簡單的 MCP Client 封裝類別
    """
    
    def __init__(self, server_url: str = "https://mcp-api.op.gg/mcp"):
        self.server_url = server_url
        self.server_params = StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "supergateway",
                "--streamableHttp",
                server_url
            ],
            env=None
        )
    
    async def list_tools(self):
        """
        列出所有可用的工具
        """
        async with stdio_client(self.server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                tools_result = await session.list_tools()
                return tools_result.tools
    
    async def call_tool(self, tool_name: str, arguments: dict):
        """
        呼叫指定的工具
        
        Args:
            tool_name: 工具名稱
            arguments: 工具參數
        
        Returns:
            工具執行結果
        """
        async with stdio_client(self.server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                result = await session.call_tool(tool_name, arguments=arguments)
                return result
    
    async def get_summoner_profile(self, region: str, summoner_id: str, game_type: str = "SOLORANKED"):
        """
        便利方法：查詢召喚師資料
        """
        return await self.call_tool(
            "lol_get_summoner_profile",
            {
                "region": region,
                "summoner_id": summoner_id,
                "game_type": game_type
            }
        )
    
    async def list_champions(self, region: str = "kr"):
        """
        便利方法：查詢英雄列表
        """
        return await self.call_tool(
            "lol_list_champions",
            {"region": region}
        )
    
    async def get_champion_analysis(self, region: str, champion_id: int, lane: str):
        """
        便利方法：查詢英雄分析
        """
        return await self.call_tool(
            "lol_get_champion_analysis",
            {
                "region": region,
                "champion_id": champion_id,
                "lane": lane
            }
        )

# 創建 client 實例
client = SimpleMCPClient()
print("✅ SimpleMCPClient 已創建")

## 5. 完整的 MCP Client 類別封裝

In [ ]:
async def call_mcp_tool_example():
    """
    範例：使用 MCP client 呼叫工具
    """
    server_params = StdioServerParameters(
        command="npx",
        args=[
            "-y",
            "supergateway",
            "--streamableHttp",
            "https://mcp-api.op.gg/mcp"
        ],
        env=None
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # 範例 1: 查詢召喚師資料
            print("🔍 查詢召喚師資料...\n")
            
            result = await session.call_tool(
                "lol_get_summoner_profile",
                arguments={
                    "region": "kr",
                    "summoner_id": "Hide on bush",
                    "game_type": "SOLORANKED"
                }
            )
            
            print("📊 結果:")
            print(result)
            print()
            
            # 範例 2: 查詢英雄列表
            print("\n🎮 查詢英雄列表...\n")
            
            result2 = await session.call_tool(
                "lol_list_champions",
                arguments={
                    "region": "kr"
                }
            )
            
            print("📊 結果:")
            print(result2)

# 執行範例 - 取消下面這行的註解來執行
# await call_mcp_tool_example()

## 4. 呼叫 MCP 工具範例

In [ ]:
async def create_mcp_client():
    """
    建立並初始化 MCP client，連接到 OP.GG MCP server
    """
    # 設定 server 參數 (使用 npx supergateway 連接到 OP.GG)
    server_params = StdioServerParameters(
        command="npx",
        args=[
            "-y",
            "supergateway",
            "--streamableHttp",
            "https://mcp-api.op.gg/mcp"
        ],
        env=None
    )
    
    # 建立 stdio client 和 session
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # 初始化連接
            await session.initialize()
            
            print("✅ 成功連接到 MCP server!")
            
            # 列出所有可用的工具
            tools_result = await session.list_tools()
            tools = tools_result.tools
            
            print(f"\n📦 可用工具數量: {len(tools)}\n")
            
            for i, tool in enumerate(tools, 1):
                print(f"{i}. {tool.name}")
                print(f"   描述: {tool.description}")
                print()
            
            return session, tools

# 執行 - 取消下面這行的註解來執行
# session, tools = await create_mcp_client()

## 3. 建立 MCP Client 連接到 OP.GG MCP Server

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

## 2. 導入必要的模組

In [ ]:
# 安裝 fastmcp 和 mcp (如果還沒安裝)
!pip install fastmcp mcp

## 1. 安裝必要套件

首先確保已安裝 fastmcp 和 mcp

# FastMCP Client 範例

這個 notebook 展示如何使用 fastmcp 建立一個 MCP client 來連接 MCP server